In [6]:
import json
import os

ABLATION STRING[i] = 1 MEANS THAT WE ARE USING THE I'TH FEATURE FROM THIS LIST OF FEATURES:<br>
[patch size , scanNet_ubiq , scanNet_protein , pesto_protein , pesto_dna_rna , pesto_ion , pesto_ligand , pesto_lipid , average_plddt]

In [7]:
hypothesis = 'pesto_protein_scores_ablation'

In [8]:
ablation_strings = [  
    # we're going over each option of: scannet_ubiq, scannet_protein, pesto_protein.
    # always using patch_size & average_plddt.
    # never using extra pesto scores.
    '100000001',  # 000: only patch_size and average_plddt
    '100100001',  # 001: regular + pesto protein - scannet protein - scannet ubiquitin
    '101000001',  # 010: regular - scannet ubiquitin 
    '101100001',  # 011: regular + pesto protein - scannet ubiquitin
    '110000001',  # 100: regular - scannet protein
    '110100001',  # 101: regular + pesto prtoein - scannet protein
    '111000001',  # 110: regular
    '111100001',  # 111: regular + pesto protein
]

In [9]:
base_configuration = {
    'hypothesis': hypothesis,
    'experiment': 'reproduce',  # will change below
    'data': {
        'path': '/home/iscb/wolfson/doririmon/home/order/ubinet/repo/ubinet/datasets/patch_to_score/data_for_training/03_04_with_pesto/folds_training_dicts.pkl',
        'use_pesto': True,
        'ablation_string': '111111111',  # will change below
        'max_folds': None
    },
    'model': {
        'name': 'patch_to_score_original',
        'kwargs': {
            'input_shape': (10, 9),
            'n_layers': 5,
            'm_a': 1024,
            'm_b': 1024,
            'm_c': 1024,
            'max_number_of_patches': 10
        }
    },
    'compile': {
        'optimizer': {
            'name': 'adam',
            'kwargs': {'learning_rate': 1e-3}
        },
        'loss': {
            'name': 'binary_cross_entropy',
            'kwargs': {}
        }
    },
    'fit': {
        'epochs': 200,
        'batch_size': 200,
        'verbose': 1,
        'n_early_stopping_epochs': 12
    }
}

In [10]:
for ablation_string in ablation_strings:
    experiment = ablation_string
    train_configuration = base_configuration
    train_configuration['data']['ablation_string'] = ablation_string
    train_configuration['model']['kwargs']['input_shape'] = (10, ablation_string.count('1'))
    train_configuration['experiment'] = experiment
    dir_path = f'../data/{hypothesis}'
    os.makedirs(dir_path, exist_ok=True)
    with open(f'{dir_path}/{experiment}.json', 'w') as f:
        json.dump(train_configuration, f)